In [1]:
import pandas as pd
import requests
from datetime import datetime, date
import os

In [2]:
APIKEY = None
with open('apikey.txt', 'r') as _apikey:
    APIKEY = _apikey.read()
    _apikey.close()

https://www.alphavantage.co/documentation/

Example: 

https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=FOREX:EUR&time_from=20220310T0130&time_to=20220410T0130&limit=100&apikey=

https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=FOREX:EUR&time_from=20220310T0130&time_to=20220510T0130&topics=finacail_market,earnings&limit=10&apikey=

In [3]:
API_URL_PRICE_DAILY = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=EUR&to_symbol=USD&outputsize=full&apikey={APIKEY}"

In [4]:
with requests.Session() as s:
    download = s.get(API_URL_PRICE_DAILY)
    df_price = pd.DataFrame(download.json()['Time Series FX (Daily)']).T
    df_price.columns = [i.split()[1] for i in df_price.columns.values]
    df_price.sort_index(inplace=True)
df_price.to_csv(f"data/APLHAVANTAGE_EURUSD.csv")
df_price

,open,high,low,close
2004-04-16,1.19720,1.20430,1.19200,1.19930
2004-04-19,1.20150,1.20770,1.19930,1.20010
2004-04-20,1.20150,1.20150,1.18180,1.18420
2004-04-21,1.18610,1.19140,1.18120,1.18200
2004-04-22,1.18280,1.19280,1.17760,1.19210
...,...,...,...,...
2023-06-09,1.07812,1.07851,1.07420,1.07460
2023-06-12,1.07447,1.07901,1.07329,1.07565
2023-06-13,1.07568,1.08235,1.07557,1.07887
2023-06-14,1.07890,1.08640,1.07730,1.08311


In [23]:
dates = df_price.index.values
dates_for_news = [i.replace('-','')+'T2350' for i in dates]

['20040416T2350',
 '20040419T2350',
 '20040420T2350',
 '20040421T2350',
 '20040422T2350',
 '20040423T2350',
 '20040426T2350',
 '20040427T2350',
 '20040428T2350',
 '20040429T2350',
 '20040430T2350',
 '20040503T2350',
 '20040504T2350',
 '20040505T2350',
 '20040506T2350',
 '20040507T2350',
 '20040510T2350',
 '20040511T2350',
 '20040512T2350',
 '20040513T2350',
 '20040514T2350',
 '20040517T2350',
 '20040518T2350',
 '20040519T2350',
 '20040520T2350',
 '20040521T2350',
 '20040524T2350',
 '20040525T2350',
 '20040526T2350',
 '20040527T2350',
 '20040528T2350',
 '20040531T2350',
 '20040601T2350',
 '20040602T2350',
 '20040603T2350',
 '20040604T2350',
 '20040607T2350',
 '20040608T2350',
 '20040609T2350',
 '20040610T2350',
 '20040611T2350',
 '20040614T2350',
 '20040615T2350',
 '20040616T2350',
 '20040617T2350',
 '20040618T2350',
 '20040621T2350',
 '20040622T2350',
 '20040623T2350',
 '20040624T2350',
 '20040625T2350',
 '20040628T2350',
 '20040629T2350',
 '20040630T2350',
 '20040701T2350',
 '20040702

In [6]:
link = "https://www.alphavantage.co/query?"
func = "function=NEWS_SENTIMENT"
tickers="tickers=FOREX:EUR"
tickers="tickers=AAPL"
time_from = "time_from=20220310T0130" 
time_to = "time_to=20220410T0130"
url_apikey = "apikey={APIKEY}"

API_URL_NEWS = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=FOREX:EUR&time_from=20220310T0130&time_to=20220410T0130&apikey={APIKEY}"
API_URL_NEWS = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&time_from=20220310T0130&time_to=20220410T0130&apikey={APIKEY}"


In [7]:
def parse_news_data(_urlNews: str):
    with requests.Session() as s:
        download = s.get(_urlNews)
        df_news = pd.json_normalize(download.json()['feed'])
        # df_news['authors_clear'] = df_news.apply(lambda x: x.authors, axis=1)
    return df_news

In [8]:
df = parse_news_data(API_URL_NEWS)

In [22]:
df_short = df[['title', 'url', 'time_published', 'source_domain']].copy()
df_short['time_published'] = df_short['time_published'].apply(lambda x: datetime.strptime(x, '%Y%m%dT%H%M%S'))
df_short.sort_values(by='time_published', ascending=True, inplace=True)
df_short.reset_index(inplace=True, drop=True)
df_short

,title,url,time_published,source_domain
0,HP Stock Jumps As Warren Buffett Stake Shows V...,https://www.investors.com/news/technology/hpq-...,2022-04-07 16:28:00,www.investors.com
1,"Google In 1996, Apple In 2010: What Does The 1...",https://www.benzinga.com/markets/cryptocurrenc...,2022-04-07 17:28:06,www.benzinga.com
2,"Google In 1996, Apple In 2010: What Does The 1...",https://www.benzinga.com/node/26511633,2022-04-07 17:28:06,www.benzinga.com
3,Here's Why Warren Buffett Bought HPQ Stock,https://www.kiplinger.com/investing/stocks/604...,2022-04-07 18:19:30,www.kiplinger.com
4,10 Stocks That Moved From Thursday's CNBC's 'F...,https://www.benzinga.com/trading-ideas/movers/...,2022-04-07 18:42:22,www.benzinga.com
5,Amazon Follows Apple and Tesla With a Big Move,https://www.thestreet.com/investing/amazon-fol...,2022-04-07 18:53:00,www.thestreet.com
6,Apple Analyst Warns Of 'Considerable Risk' For...,https://www.benzinga.com/analyst-ratings/analy...,2022-04-07 19:41:35,www.benzinga.com
7,SHAREHOLDER ALERT: Pomerantz Law Firm Investig...,https://www.benzinga.com/pressreleases/22/04/g...,2022-04-07 19:42:07,www.benzinga.com
8,Indexes Close Higher After Digesting Fed Minut...,https://www.investors.com/market-trend/stock-m...,2022-04-07 20:46:00,www.investors.com
9,4 Top Computer Hardware Stocks To Watch Right Now,https://stockmarket.com/featured/4-top-compute...,2022-04-07 21:00:59,stockmarket.com


In [10]:
df_short.source_domain.unique()

array(['www.investors.com', 'www.benzinga.com', 'www.kiplinger.com',
       'www.thestreet.com', 'stockmarket.com', 'www.forbes.com',
       'www.zacks.com', 'www.business-standard.com', 'www.reuters.com',
       'www.marketwatch.com', 'stocknews.com', 'www.fool.com',
       'www.barrons.com', 'www.moneycontrol.com',
       'markets.businessinsider.com'], dtype=object)

In [11]:
from bs4 import BeautifulSoup
import requests

In [12]:
#df_short['status_code'] = df_short.url.apply(lambda x: requests.get(x).status_code)

In [13]:
#df_short[df_short.status_code == 200]

In [14]:
url = df_short.url[1]
page = requests.get(url)
print(page.status_code)
soup = BeautifulSoup(page.text, "lxml")
soup.prettify()
url


200


'https://www.benzinga.com/markets/cryptocurrency/22/04/26511633/google-in-1996-apple-in-2010-what-does-the-14-year-investment-thesis-mean-for-crypto-in-20'

In [15]:
text_raw = soup.findAll(class_="core-block") #benzinga

In [16]:
text_to_print = ""
for i in text_raw:
    text_to_print += i.text

text_to_print
    

'Mark Yusko, founder, CIO and CEO of Morgan Creek Capital Management, explained how he gained his experience in the world of investing on Benzinga\'s podcast "The Long Run Show."Mark YuskoMorgan Creek Capital ManagementThe Long Run ShowListen to the full interview here. here. Despite extraordinary returns from early-stage investments, the most compelling result of Yusko\'s lengthy investing career might be the 14-year investing thesis he has developed. He maintains that every 14 years an innovation comes along that completely changes the technological landscape.14-Year Approach: "If you go back to 1954, there was this innovation out in Boston, outside of route 128 around computing and suddenly companies could have computers," Yusko said, referencing Wang Laboratories.\xa014-Year Approach:In 1968, innovation ramped up in Silicon Valley and companies like\xa0Intel Corp INTC and eventually Fairchild Semiconductor were formed to make microchips for computers, he explained.Intel CorpAbout 1

In [17]:
text_raw

[<p class="core-block"><strong class="core-block">Mark Yusko</strong>, founder, CIO and CEO of <strong class="core-block">Morgan Creek Capital Management</strong>, explained how he gained his experience in the world of investing on Benzinga's podcast "<a class="core-block" href="https://www.benzinga.com/podcasts/the-14-year-investment-pattern-with-mark-yusko" target="_blank">The Long Run Show</a>."</p>,
 <strong class="core-block">Mark Yusko</strong>,
 <strong class="core-block">Morgan Creek Capital Management</strong>,
 <a class="core-block" href="https://www.benzinga.com/podcasts/the-14-year-investment-pattern-with-mark-yusko" target="_blank">The Long Run Show</a>,
 <p class="core-block">Listen to the full interview <a class="core-block" href="https://podcasts.apple.com/us/podcast/the-14-year-investment-pattern-with-mark-yusko/id1588788159?i=1000556298236" target="_blank">here. </a></p>,
 <a class="core-block" href="https://podcasts.apple.com/us/podcast/the-14-year-investment-pattern

In [18]:
df_short.url[3]

'https://www.kiplinger.com/investing/stocks/604520/heres-why-warren-buffett-bought-hpq-stock'

In [19]:
pages = []
for i, url in enumerate(df_short.url):
    page = requests.get(url)
    if page.status_code == 200:
       print(df_short.iloc[i])
       pages.append([df_short.iloc[i], page]) 

# url = df_short.url[1]
# page = requests.get(url)
# soup = BeautifulSoup(page.text, "lxml")
# soup.prettify()
# url

title             Google In 1996, Apple In 2010: What Does The 1...
url               https://www.benzinga.com/markets/cryptocurrenc...
time_published                                  2022-04-07 17:28:06
source_domain                                      www.benzinga.com
Name: 1, dtype: object
title             Google In 1996, Apple In 2010: What Does The 1...
url                          https://www.benzinga.com/node/26511633
time_published                                  2022-04-07 17:28:06
source_domain                                      www.benzinga.com
Name: 2, dtype: object
title                    Here's Why Warren Buffett Bought HPQ Stock
url               https://www.kiplinger.com/investing/stocks/604...
time_published                                  2022-04-07 18:19:30
source_domain                                     www.kiplinger.com
Name: 3, dtype: object
title             10 Stocks That Moved From Thursday's CNBC's 'F...
url               https://www.benzinga.com/trad

In [20]:
def parse_benzinga(page):
    soup = BeautifulSoup(page.text, "lxml")
    soup.prettify()
    text_raw = soup.findAll(class_="core-block")
    return [i.text for i in text_raw]
    
def parse_kiplinger(page):
    soup = BeautifulSoup(page.text, "lxml")
    soup.prettify()
    text_raw = soup.find(class_="article__body").findAll('p')
    return [i.text for i in text_raw]

def parse_stockmarket(page):
    soup = BeautifulSoup(page.text, "lxml")
    soup.prettify()
    text_raw = soup.find(class_="entry-content").findAll('p')
    return [i.text for i in text_raw]

def parse_forbes(page):    
    soup = BeautifulSoup(page.text, "lxml")
    soup.prettify()
    text_raw = soup.find(class_="article-body fs-article fs-responsive-text current-article").findAll('p')
    return [i.text for i in text_raw]

def parse_business_standard(page):
    soup = BeautifulSoup(page.text, "lxml")
    soup.prettify()
    text_raw = soup.find(class_="storycontent").findAll('p')
    return [i.text for i in text_raw]

def parse_reuters(page):
    soup = BeautifulSoup(page.text, "lxml")
    soup.prettify()
    text_raw = soup.find(class_="article-body__content__17Yit").findAll('p')
    return [i.text for i in text_raw]

def parse_marketwatch(page):
    soup = BeautifulSoup(page.text, "lxml")
    soup.prettify()
    text_raw = soup.find(class_="article__content").findAll('p')
    return [i.text for i in text_raw]

def parse_stocknews(page):
    soup = BeautifulSoup(page.text, "lxml")
    soup.prettify()
    text_raw = soup.find(class_="post_content").findAll('p')
    return [i.text for i in text_raw]

def parse_fool(page):
    soup = BeautifulSoup(page.text, "lxml")
    soup.prettify()
    text_raw = soup.find(class_="tailwind-article-body").findAll('p')
    return [i.text for i in text_raw]

def parse_moneycontrol(page):
    soup = BeautifulSoup(page.text, "lxml")
    soup.prettify()
    text_raw = soup.find(class_="article-p-wrap").findAll('p')
    return [i.text for i in text_raw]

def parse_markets_businessinsider(page):
    soup = BeautifulSoup(page.text, "lxml")
    soup.prettify()
    text_raw = soup.find(class_="col-xs-12 news-content no-padding").findAll('p')
    return [i.text for i in text_raw]

# def parse_(page):
#     soup = BeautifulSoup(page.text, "lxml")
#     soup.prettify()
#     text_raw = soup.find(class_="").findAll('p')
#     return [i.text for i in text_raw]

In [21]:
parsers = {'www.benzinga.com' : parse_benzinga,
           'www.kiplinger.com' : parse_kiplinger,
           'stockmarket.com' : parse_stockmarket,
           'www.forbes.com' : parse_forbes,
           'www.business-standard.com' : parse_business_standard,
           'www.reuters.com' : parse_reuters,
           'www.marketwatch.com' : parse_marketwatch,
           'stocknews.com' : parse_stocknews,
           'www.fool.com' : parse_fool,
           'www.moneycontrol.com' : parse_moneycontrol,
           'markets.businessinsider.com' : parse_markets_businessinsider}

os.makedirs('to_plot', exist_ok=True)

for page_ in pages:
    info = page_[0]
    
    title = info['title']
    source_domain = info['source_domain']
    link = info['url']
    time_published = info['time_published']
    
    print(source_domain, title, link, sep='\n')
    
    parser = parsers[source_domain] 
    text_raw = parser(page=page_[1])
    
    text_to_print = ""
    for i in text_raw:
        text_to_print += i

    

    print(text_to_print.replace('\n', ''))
    print('\n')
    

www.benzinga.com
Google In 1996, Apple In 2010: What Does The 14-Year Investment Thesis Mean For Crypto In 2024?
https://www.benzinga.com/markets/cryptocurrency/22/04/26511633/google-in-1996-apple-in-2010-what-does-the-14-year-investment-thesis-mean-for-crypto-in-20
Mark Yusko, founder, CIO and CEO of Morgan Creek Capital Management, explained how he gained his experience in the world of investing on Benzinga's podcast "The Long Run Show."Mark YuskoMorgan Creek Capital ManagementThe Long Run ShowListen to the full interview here. here. Despite extraordinary returns from early-stage investments, the most compelling result of Yusko's lengthy investing career might be the 14-year investing thesis he has developed. He maintains that every 14 years an innovation comes along that completely changes the technological landscape.14-Year Approach: "If you go back to 1954, there was this innovation out in Boston, outside of route 128 around computing and suddenly companies could have computers," 